<a href="https://colab.research.google.com/github/CaptainOuch/Text_Improvement_Engine/blob/main/Text_Improvement_Engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [145]:
!pip install openai

In [4]:
import openai

In [5]:
api_key = 'PasteYourKey'
openai.api_key = api_key

In [87]:
def split_text_into_word_pairs(text):
    words = text.split()
    word_pairs = []

    for i in range(len(words) - 2):
        pair = words[i] + ' ' + words[i + 1]
        word_pairs.append(pair)

        pair = words[i] + ' ' + words[i + 1] + ' ' + words[i + 2]
        word_pairs.append(pair)

    pair = words[len(words) - 1] + ' ' + words[len(words)]

    word_pairs.append(pair)

    return word_pairs

input_text = "Let's jump together friend!"
result = split_text_into_word_pairs(input_text)
print(result)

["Let's jump", "Let's jump together", 'jump together', 'jump together friend!']


In [133]:
def create_embedding(terms_list):
    resp = openai.Embedding.create(
      input=terms_list,
      engine="text-similarity-curie-001")

    return resp

In [15]:
def remove_non_alphanumeric_chars(text):
    cleaned_text = ""

    # Проходим по каждому символу в исходном тексте
    for char in text:
        if char.isalnum() or char.isspace():
            cleaned_text += char

    return cleaned_text

input_text = "This is an example text with 123 numbers and special characters !@#"
cleaned_text = remove_non_alphanumeric_chars(input_text)
print(cleaned_text)


This is an example text with 123 numbers and special characters 


In [92]:
standardised_terms = [
    "Optimal performance",
    "Utilise resources",
    "Enhance productivity",
    "Conduct an analysis",
    "Maintain a high standard",
    "Implement best practices",
    "Ensure compliance",
    "Streamline operations",
    "Foster innovation",
    "Drive growth",
    "Leverage synergies",
    "Demonstrate leadership",
    "Exercise due diligence",
    "Maximize stakeholder value",
    "Prioritise tasks",
    "Facilitate collaboration",
    "Monitor performance metrics",
    "Execute strategies",
    "Gauge effectiveness",
    "Champion change"
]

In [134]:
terms_emb = create_embedding(standardised_terms)
len(terms_emb['data'])

20

In [94]:
meeting_text = """
In today's meeting, we discussed a variety of issues affecting our department.
The weather was unusually sunny, a pleasant backdrop to our serious discussions.
We came to the consensus that we need to do better in terms of performance.
Sally brought doughnuts, which lightened the mood.
It's important to make good use of what we have at our disposal.
During the coffee break, we talked about the upcoming company picnic.
We should aim to be more efficient and look for ways to be more creative in our daily tasks.
Growth is essential for our future, but equally important is building strong relationships with our team members.
As a reminder, the annual staff survey is due next Friday.
Lastly, we agreed that we must take time to look over our plans carefully and consider all angles before moving forward.
On a side note, David mentioned that his cat is recovering well from surgery.
"""

In [126]:
cleaned_text = remove_non_alphanumeric_chars(meeting_text)
splitted_text = split_text_into_word_pairs(cleaned_text)
print(splitted_text)

['In todays', 'In todays meeting', 'todays meeting', 'todays meeting we', 'meeting we', 'meeting we discussed', 'we discussed', 'we discussed a', 'discussed a', 'discussed a variety', 'a variety', 'a variety of', 'variety of', 'variety of issues', 'of issues', 'of issues affecting', 'issues affecting', 'issues affecting our', 'affecting our', 'affecting our department', 'our department', 'our department The', 'department The', 'department The weather', 'The weather', 'The weather was', 'weather was', 'weather was unusually', 'was unusually', 'was unusually sunny', 'unusually sunny', 'unusually sunny a', 'sunny a', 'sunny a pleasant', 'a pleasant', 'a pleasant backdrop', 'pleasant backdrop', 'pleasant backdrop to', 'backdrop to', 'backdrop to our', 'to our', 'to our serious', 'our serious', 'our serious discussions', 'serious discussions', 'serious discussions We', 'discussions We', 'discussions We came', 'We came', 'We came to', 'came to', 'came to the', 'to the', 'to the consensus', '

In [135]:
text_pair_emb = create_embedding(splitted_text)
len(text_pair_emb['data'])

300

In [21]:
import numpy as np

In [103]:
embedding_a = terms_emb['data'][0]['embedding']
embedding_b = text_pair_emb['data'][0]['embedding']
np.dot(embedding_a, embedding_b)

0.7546026551915956

In [116]:
import time

def compute_dot_products(
    pair_emb: dict,
    terms_emb: dict,
    splitted_text: list,
    standardised_terms: list,
    threshold: float = 0.85
    ) -> list:
  final_pairs = {}

  start_time = time.time()

  for i in range(len(splitted_text)):
    for j in range(len(standardised_terms)):
      pair_emb_i = pair_emb['data'][i]['embedding']
      terms_emb_j = terms_emb['data'][j]['embedding']
      dot_product = np.dot(pair_emb_i, terms_emb_j)

      if dot_product > threshold:
        final_pairs[splitted_text[i]] = [standardised_terms[j], dot_product]

  end_time = time.time()
  execution_time = end_time - start_time

  return final_pairs, execution_time


In [136]:
result = compute_dot_products(
    pair_emb=text_pair_emb,
    terms_emb=terms_emb,
    splitted_text=splitted_text,
    standardised_terms=standardised_terms,
    threshold=0.85
    )

final_pairs, time_spent = zip(result)

print(final_pairs[0])
print("Execution time:", time_spent[0], "seconds")

{'terms of performance': ['Optimal performance', 0.8615814624019358], 'of performance': ['Optimal performance', 0.892997306396373], 'make good use': ['Utilise resources', 0.8677328491585122], 'be more efficient': ['Enhance productivity', 0.8589709161108788], 'Growth is': ['Drive growth', 0.8659110568396847]}
Execution time: 4.222826957702637 seconds
